In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import pandas as pd
import ipywidgets as widgets
from IPython.display import display

widgets.interact(lambda x: f'你選擇了：{x} 次', x=widgets.IntSlider(value=3, min=1, max=5))

# 讀取新的 Excel 資料
df = pd.read_excel("data (20250101-20250815) no name.xlsx")

# 處理日期與時間
df['date'] = pd.to_datetime(df['date']).dt.date
df['time'] = pd.to_datetime(df['time']).dt.time
df['datetime'] = pd.to_datetime(df['date'].astype(str) + ' ' + df['time'].astype(str))
df['hour'] = df['datetime'].dt.hour + df['datetime'].dt.minute / 60

# 生成 week_range 的映射
week_range_map = df.drop_duplicates(subset=['week'])[['week', 'range']].set_index('week')['range'].to_dict()

# 確認更新的週次與週期範圍
df[['week', 'range']].drop_duplicates().sort_values(by='week').reset_index(drop=True)


interactive(children=(IntSlider(value=3, description='x', max=5, min=1), Output()), _dom_classes=('widget-inte…

,week,range
0,2025-W01,2024-12-30 ~ 2025-01-05
1,2025-W02,2025-01-06 ~ 2025-01-12
2,2025-W03,2025-01-13 ~ 2025-01-19
3,2025-W04,2025-01-20 ~ 2025-01-26
4,2025-W06,2025-02-03 ~ 2025-02-09
5,2025-W07,2025-02-10 ~ 2025-02-16
6,2025-W08,2025-02-17 ~ 2025-02-23
7,2025-W09,2025-02-24 ~ 2025-03-02
8,2025-W10,2025-03-03 ~ 2025-03-09
9,2025-W11,2025-03-10 ~ 2025-03-16


In [2]:
# 每人每週運動幾天（不重複同日）
weekly_counts = df.groupby(['week', 'name', 'date']).size().reset_index(name='count')
weekly_visits = weekly_counts.groupby(['week', 'name']).size().reset_index(name='visits_per_week')

# 篩選3~5次
summary = weekly_visits[weekly_visits['visits_per_week'].isin([3, 4, 5])]

# 加入 week_range，使用 .copy() 避免警告
summary = summary.copy()
week_range_map = df.drop_duplicates(subset=['week'])[['week', 'range']].set_index('week')['range'].to_dict()
summary.loc[:, 'week_range'] = summary['week'].map(week_range_map)

# 整理
weekly_result = summary.groupby(['week', 'week_range', 'visits_per_week'])['name']    .agg(['count', lambda x: ', '.join(sorted(x))])    .reset_index().rename(columns={'count': '人數', '<lambda_0>': '編號'})
weekly_result


,week,week_range,visits_per_week,人數,編號
0,2025-W02,2025-01-06 ~ 2025-01-12,3,6,"CZ, DB, EN, EX, FQ, HQ"
1,2025-W02,2025-01-06 ~ 2025-01-12,4,1,CP
2,2025-W03,2025-01-13 ~ 2025-01-19,3,4,"EP, EX, F, FQ"
3,2025-W03,2025-01-13 ~ 2025-01-19,4,2,"CP, GM"
4,2025-W04,2025-01-20 ~ 2025-01-26,3,6,"CZ, DB, DG, FQ, GQ, L"
...,...,...,...,...,...
72,2025-W32,2025-08-04 ~ 2025-08-10,3,5,"BL, CP, EA, EJ, GT"
73,2025-W32,2025-08-04 ~ 2025-08-10,4,10,"AN, CB, DA, DZ, EB, EK, EU, FM, HT, O"
74,2025-W32,2025-08-04 ~ 2025-08-10,5,4,"CY, DE, FQ, HN"
75,2025-W33,2025-08-11 ~ 2025-08-17,3,9,"AK, CB, DE, FM, FQ, GD, HN, HT, IE"


In [3]:
# 篩選 visits_per_week 為 1 或 2
summary_low = weekly_visits[weekly_visits['visits_per_week'].isin([1, 2])].copy()

# 加入 week_range
summary_low['week_range'] = summary_low['week'].map(week_range_map)

# 整理顯示格式
weekly_result_low = (
    summary_low.groupby(['week', 'week_range', 'visits_per_week'])['name']
    .agg(['count', lambda x: ', '.join(sorted(x))])
    .reset_index()
)

# 欄位重新命名
weekly_result_low.columns = ['week', 'week_range', 'visits_per_week', '人數', '編號']

# 顯示結果
weekly_result_low


,week,week_range,visits_per_week,人數,編號
0,2025-W01,2024-12-30 ~ 2025-01-05,1,7,"AE, BE, CA, CZ, GI, GM, II"
1,2025-W01,2024-12-30 ~ 2025-01-05,2,3,"BM, CP, FQ"
2,2025-W02,2025-01-06 ~ 2025-01-12,1,46,"AF, AH, AI, AS, AV, AZ, BA, BD, BE, BF, BM, BU..."
3,2025-W02,2025-01-06 ~ 2025-01-12,2,16,"AJ, BQ, CA, CU, DF, DQ, DR, DS, EP, EQ, ER, G,..."
4,2025-W03,2025-01-13 ~ 2025-01-19,1,35,"AC, AJ, AW, CH, CI, CN, CQ, DC, DJ, DQ, DR, DX..."
...,...,...,...,...,...
59,2025-W31,2025-07-28 ~ 2025-08-03,2,8,"AN, CA, CB, CY, FH, FN, GD, O"
60,2025-W32,2025-08-04 ~ 2025-08-10,1,23,"AI, AM, BJ, BS, BX, BY, BZ, CA, DF, DM, EH, FC..."
61,2025-W32,2025-08-04 ~ 2025-08-10,2,4,"AK, AO, FH, IE"
62,2025-W33,2025-08-11 ~ 2025-08-17,1,19,"AW, BO, BT, BX, BZ, CV, DF, DH, DM, DN, DZ, ED..."


In [4]:
# 合併高頻與低頻資料
weekly_result_all = pd.concat([weekly_result, weekly_result_low], ignore_index=True)

# 確保 visits_per_week 為 int 型別（slider 才能正確操作）
weekly_result_all['visits_per_week'] = weekly_result_all['visits_per_week'].astype(int)

# 排序資料（可選）
weekly_result_all = weekly_result_all.sort_values(by=['week', 'visits_per_week']).reset_index(drop=True)

# 下拉選單：週次
week_dropdown = widgets.Dropdown(
    options=weekly_result_all['week'].unique().tolist(),
    description='週次:',
    value=weekly_result_all['week'].unique().tolist()[0],
    layout=widgets.Layout(width='300px')
)

# 滑桿：運動次數範圍
visits_slider = widgets.IntRangeSlider(
    value=[1, 5],
    min=1,
    max=5,
    step=1,
    description='每週次數:',
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

# 顯示函式
def update_table(week, visits_range):
    filtered = weekly_result_all[
        (weekly_result_all['week'] == week) &
        (weekly_result_all['visits_per_week'] >= visits_range[0]) &
        (weekly_result_all['visits_per_week'] <= visits_range[1])
    ]
    display(filtered[['week_range', 'visits_per_week', '人數', '編號']])
    print(f"\n總人數：{filtered['人數'].sum()} 人")

# 綁定互動
widgets.interact(
    update_table,
    week=week_dropdown,
    visits_range=visits_slider
);


interactive(children=(Dropdown(description='週次:', layout=Layout(width='300px'), options=('2025-W01', '2025-W02…

In [5]:
# 將 week_range 的第一天取出作為該週的基準月份
weekly_result_all['week_start_date'] = weekly_result_all['week_range'].str.extract(r'(^\d{4}-\d{2}-\d{2})')
weekly_result_all['week_start_date'] = pd.to_datetime(weekly_result_all['week_start_date'])

# 轉換成月份格式：2025-M01、2025-M02 ...
weekly_result_all['month'] = weekly_result_all['week_start_date'].dt.strftime('%Y-M%m')

# 下拉選單：月份
month_dropdown = widgets.Dropdown(
    options=weekly_result_all['month'].unique().tolist(),
    description='月份:',
    value=weekly_result_all['month'].unique().tolist()[0],
    layout=widgets.Layout(width='300px')
)

# 滑桿：運動次數
visits_slider = widgets.IntRangeSlider(
    value=[1, 5],
    min=1,
    max=5,
    step=1,
    description='每週次數:',
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

def update_table_by_month(month, visits_range):
    filtered = weekly_result_all[
        (weekly_result_all['month'] == month) &
        (weekly_result_all['visits_per_week'] >= visits_range[0]) &
        (weekly_result_all['visits_per_week'] <= visits_range[1])
    ]
    display(filtered[['week', 'week_range', 'visits_per_week', '人數', '編號']])
    print(f"\n總人數：{filtered['人數'].sum()} 人")

widgets.interact(
    update_table_by_month,
    month=month_dropdown,
    visits_range=visits_slider
);


interactive(children=(Dropdown(description='月份:', layout=Layout(width='300px'), options=('2024-M12', '2025-M01…

In [6]:
time_slots = {
    "09:00-10:00": (9.0, 10.0),
    "10:00-11:00": (10.0, 11.0),
    "11:00-12:00": (11.0, 12.0),
    "13:30-14:30": (13.5, 14.5),
    "14:30-15:30": (14.5, 15.5),
    "15:30-16:30": (15.5, 16.5),
    "16:30-17:30": (16.5, 17.5),
}

def assign_time_slot(hour):
    for slot, (start, end) in time_slots.items():
        if start <= hour < end:
            return slot
    return None

df['time_slot'] = df['hour'].apply(assign_time_slot)

# 篩選1~8月，且不重複：同人同日同時段算一次
df_18 = df[df['datetime'].dt.month <= 8].drop_duplicates(subset=['name', 'date', 'time_slot'])

# ➤ 每月各時段
monthly_slot = df_18.groupby([df_18['datetime'].dt.month.rename('月份'), 'time_slot'])['name']    .nunique().reset_index(name='使用人數')

# ➤ 每週各時段
weekly_slot = df_18.groupby(['week', 'time_slot'])['name']    .nunique().reset_index(name='使用人數')

from IPython.display import display

# 建立 week_range 對照表
week_range_map = df.drop_duplicates(subset=['week'])[['week', 'range']] \
                   .set_index('week')['range'].to_dict()

# 加入 week_range 欄位
weekly_slot['week_range'] = weekly_slot['week'].map(week_range_map)

# ➤ 標註每週最多人時段
weekly_peak = weekly_slot.loc[weekly_slot.groupby('week')['使用人數'].idxmax()]
weekly_peak['高峰'] = '⭐最多人'

# ➤ 合併高峰標註
weekly_slot = weekly_slot.merge(
    weekly_peak[['week', 'time_slot', '高峰']],
    on=['week', 'time_slot'], how='left'
)
weekly_slot = weekly_slot.fillna({'高峰': ''})

# ➤ 重新調整欄位順序
weekly_slot = weekly_slot[['week', 'week_range', 'time_slot', '使用人數', '高峰']]

# ➤ 顯示完整設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# ➤ 顯示最終表格
display(weekly_slot)



,week,week_range,time_slot,使用人數,高峰
0,2025-W01,2024-12-30 ~ 2025-01-05,09:00-10:00,2,
1,2025-W01,2024-12-30 ~ 2025-01-05,10:00-11:00,5,⭐最多人
2,2025-W01,2024-12-30 ~ 2025-01-05,11:00-12:00,4,
3,2025-W01,2024-12-30 ~ 2025-01-05,14:30-15:30,1,
4,2025-W01,2024-12-30 ~ 2025-01-05,15:30-16:30,4,
5,2025-W01,2024-12-30 ~ 2025-01-05,16:30-17:30,3,
6,2025-W02,2025-01-06 ~ 2025-01-12,09:00-10:00,17,
7,2025-W02,2025-01-06 ~ 2025-01-12,10:00-11:00,28,
8,2025-W02,2025-01-06 ~ 2025-01-12,11:00-12:00,23,
9,2025-W02,2025-01-06 ~ 2025-01-12,13:30-14:30,11,
